In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
class Node:
    def __init__(self):
        self.attr = None
        self.label = None
        self.next = None

In [22]:
class C45:
    def entropy(self,y):
        E = 0
        for v in np.unique(y):
            prop = np.sum(y==v)/len(y)
            E += (-prop*np.log2(prop))
        return E
    def findBestCutPoint(self,X,y):
        d = []
        attr_val = np.unique(X)
        if len(attr_val)<=1:
            return (np.inf,None)
        for i in attr_val:
            d.append((i,np.unique(y[X==i].values)))
        # print(d)
        cut_points = []
        for i in range(len(d)-1):
            if len(d[i][1]) == 1 and len(d[i+1][1]) == 1:
                if d[i][1] != d[i+1][1]:
                    cut_points.append(d[i][0])
            else:
                cut_points.append(d[i][0])
        S = {}
        # print(cut_points)
        for c in cut_points:
            prop1 = len(y[X<=c])/len(y)
            prop2 = len(y[X>c])/len(y)
            # print(prop1,prop2)
            S[c] = prop1*self.entropy(y[X<=c])+prop2*self.entropy(y[X>c])
        print(S)
        print(f'Best Cut point: {min(S.values())} and {min(S,key=S.get)}')
        return (min(S.values()),min(S,key=S.get))
    def selectBestAttr(self,X,y,features,attr_values):
        S = self.entropy(y)
        # print(len(y))
        print(S)
        S_a = []
        cp_index = {}
        for i in range(len(features)):
            s_a = 0
            split_info = 0 
            if attr_values[features[i]].dtype == 'O':
                for v in attr_values[features[i]]:
                    y_v = y[X[features[i]] == v]
                    s_a += (len(y_v)/len(y))*self.entropy(y_v)
                    prop = len(y_v)/len(y)
                    if prop:
                        split_info += (-prop*np.log2(prop))
            else:
                (s_a,cp) = self.findBestCutPoint(X[features[i]],y)
                cp_index[i] = cp
                y_v = y[X[features[i]]<=cp]
                prop = len(y_v)/len(y)
                if prop and prop!=1:
                    split_info = -prop*np.log2(prop)-(1-prop)*np.log2(1-prop)
                elif prop == 1:
                    split_info = -prop*np.log2(prop)
                else:
                    split_info = -(1-prop)*np.log2(1-prop)
            print(split_info)
            if split_info:
                S_a.append((S-s_a)/split_info)
            else:
                S_a.append(S-s_a)
        S_a = np.array(S_a)
        print(S_a)
        best_index = S_a.argmax()
        if best_index in cp_index.keys():
            return (best_index,cp_index[best_index])
        return (best_index,None)
    def c45(self,X,y,features,attr_values):
        root = Node()
        labels = np.unique(y)
        if len(labels)==1:
            # print(X,y)
            root.label = labels[0]
            return root
        if not len(features):
            root.label = stats.mode(y)[0][0]
            return root
        attr_index,cp = self.selectBestAttr(X,y,features,attr_values)
        if X[features[attr_index]].dtype!='O' and cp == None:
            print('yes')
            root.label = stats.mode(y)[0][0]
            return root
        root.attr = features[attr_index]
        root.next = []
        if cp == None:
            for v in attr_values[root.attr]:
                X_v = X[X[root.attr] == v]
                y_v = y[X[root.attr] == v]
                # print(root.attr,v)
                # print(len(y_v))
                if not len(X_v):
                    temp = Node()
                    temp.label = stats.mode(y)[0][0]
                    root.next.append((v,temp))
                else:
                    temp = self.c45(X_v,y_v,features[0:attr_index]+features[attr_index+1:],attr_values)
                    root.next.append((v,temp))
        else:
            X_v1,X_v2 = X[X[root.attr] <= cp],X[X[root.attr] > cp]
            y_v1,y_v2 = y[X[root.attr] <= cp],y[X[root.attr] > cp]
            if not len(X_v1):
                temp = Node()
                temp.label = stats.mode(y)[0][0]
                root.next.append(((cp,0),temp))
            else:
                temp = self.c45(X_v1,y_v1,features[0:attr_index]+features[attr_index+1:],attr_values)
                root.next.append(((cp,0),temp))
            if not len(X_v2):
                temp = Node()
                temp.label = stats.mode(y)[0][0]
                root.next.append(((cp,1),temp))
            else:
                temp = self.c45(X_v2,y_v2,features[0:attr_index]+features[attr_index+1:],attr_values)
                root.next.append(((cp,1),temp))
        return root
    def fit(self,data,features,target):
        X = data[features]
        y = data[target]
        attr_values = {}
        for a in features:
            attr_values[a] = np.unique(X[a])
        # print(attr_values)
        root = self.c45(X,y,features,attr_values)
        return root
    def findPath(self,x,root):
        if root.label != None:
            return root.label
        for child in root.next:
            if type(x[root.attr]) == str:
                if child[0] == x[root.attr]:
                    ans = self.findPath(x,child[1])
                    break
            else:
                if not child[0][1] and x[root.attr] <= child[0][0]:
                    ans = self.findPath(x,child[1])
                    break
                if child[0][1] and x[root.attr] > child[0][0]:
                    ans = self.findPath(x,child[1])
                    break
        return ans
    def predict(self,root,test_X):
        y = []
        for x in test_X.index:
            for child in root.next:
                if test_X[root.attr].dtype == 'O':
                    if child[0] == test_X.loc[x][root.attr]:
                        y.append(self.findPath(test_X.loc[x],child[1]))
                        break
                else:
                    if not child[0][1] and test_X.loc[x][root.attr] <= child[0][0]:
                        y.append(self.findPath(test_X.loc[x],child[1]))
                        break
                    if child[0][1] and test_X.loc[x][root.attr] > child[0][0]:
                        y.append(self.findPath(test_X.loc[x],child[1]))
                        break
        y = np.array(y)
        return y
    def viewTree(self,root):
        queue = []
        queue.append((root,0))
        while len(queue):
            temp,depth = queue.pop(0)
            i = 1
            print(f'Depth: {depth} Attribute: {temp.attr}')
            for t in temp.next:
                if t[1].label == None:
                    print(f'[{i}: {t[0]}: Attribute: {t[1].attr}]',end="  ")
                    queue.append((t[1],depth+1))
                else:
                    print(f'[{i}: {t[0]}: Class label: {t[1].label}]',end="  ")
                i+=1
            print('\n')
    def accuracy(self,true_y,pred_y):
        acc = np.sum(true_y == pred_y)/len(true_y)
        return acc*100

In [9]:
data = pd.read_csv('playtennis.csv')

In [10]:
tree = C45()

In [11]:
features = ['Outlook','Temperature','Humidity','Wind']
target = 'PlayTennis'
root = tree.fit(data,features,target)

0.9402859586706311
1.5774062828523454
1.5566567074628228
1.0
0.9852281360342515
[0.15642756 0.01877265 0.1518355  0.04884862]
0.9709505944546686
0.9709505944546686
0.9709505944546686
0.9709505944546686
[0.02057066 0.02057066 1.        ]
0.9709505944546686
1.5219280948873621
0.9709505944546686
0.9709505944546686
[0.37514952 1.         0.02057066]


In [12]:
tree.viewTree(root)

Depth: 0 Attribute: Outlook
[1: Overcast: Class label: Yes]  [2: Rain: Attribute: Wind]  [3: Sunny: Attribute: Humidity]  

Depth: 1 Attribute: Wind
[1: Strong: Class label: No]  [2: Weak: Class label: Yes]  

Depth: 1 Attribute: Humidity
[1: High: Class label: No]  [2: Normal: Class label: Yes]  



In [13]:
cars = pd.read_csv('car2.csv')

In [14]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cars,test_size=0.25,random_state=0)

In [23]:
tree2 = C45()

In [24]:
feat = ['buying','maint','doors','persons','lug_boot','safety']
tar = 'target'
root = tree2.fit(train,feat,tar)

1.1905659226383394
1.9999312830459477
1.9986798011680582
1.999791287348977
1.5849289657499317
1.5848827062560256
1.5844573535722146
[0.04545163 0.03629352 0.00211369 0.13701845 0.01958171 0.16745085]
1.6043412313456216
1.9991719294984234
1.9959436702868514
1.99880340823794
1.5840973906597133
1.584052107462226
[0.10972866 0.06215642 0.00515639 0.3159917  0.0700378 ]
1.65144272472017
1.9968642869590907
1.992523291312304
1.9953992478270552
1.584656501219507
[0.24886015 0.18586951 0.01725696 0.10510756]
0.8112781244591328
1.9287289082033152
1.9620310230081115
1.5835382325345573
[0.42062838 0.00828749 0.00283973]
1.555749684676997
1.9612325352153852
1.9759583325044725
1.580982641705794
[0.37992691 0.03405059 0.28675231]
0.9940302114769565
1.9808259362290785
1.5726236638951638
[0.03073419 0.40085487]
1.0
2.0
[0.5]
0.9852281360342515
1.5566567074628228
1.5566567074628228
[0.01300493 0.44936937]
1.0
1.0
[1.]
0.9910760598382222
1.8910611120726526
1.584962500721156
[0.08284059 0.43217265]
0.9182

In [18]:
tree2.viewTree(root)

Depth: 0 Attribute: safety
[1: high: Attribute: persons]  [2: low: Class label: unacc]  [3: med: Attribute: persons]  

Depth: 1 Attribute: persons
[1: 2: Class label: unacc]  [2: 4: Attribute: buying]  [3: more: Attribute: buying]  

Depth: 1 Attribute: persons
[1: 2: Class label: unacc]  [2: 4: Attribute: buying]  [3: more: Attribute: lug_boot]  

Depth: 2 Attribute: buying
[1: high: Attribute: maint]  [2: low: Attribute: maint]  [3: med: Attribute: maint]  [4: vhigh: Attribute: maint]  

Depth: 2 Attribute: buying
[1: high: Attribute: maint]  [2: low: Attribute: lug_boot]  [3: med: Attribute: maint]  [4: vhigh: Attribute: maint]  

Depth: 2 Attribute: buying
[1: high: Attribute: lug_boot]  [2: low: Attribute: maint]  [3: med: Attribute: maint]  [4: vhigh: Attribute: maint]  

Depth: 2 Attribute: lug_boot
[1: big: Attribute: maint]  [2: med: Attribute: buying]  [3: small: Attribute: buying]  

Depth: 3 Attribute: maint
[1: high: Class label: acc]  [2: low: Class label: acc]  [3: med:

In [25]:
test_X = test.iloc[:,:-1]

In [26]:
pred_y = tree2.predict(test_X=test_X,root=root)

In [27]:
tree2.accuracy(test.iloc[:,-1].values,pred_y)

93.28703703703704

In [28]:
t_pred_y = tree2.predict(root,train.iloc[:,:-1])

In [29]:
tree2.accuracy(train.iloc[:,-1].values,t_pred_y)

100.0

In [30]:
data = pd.read_csv('dtr3_2.csv')

In [34]:
tree4 = C45()

In [35]:
features = ['Outlook','Temperature','Humidity','Wind']
target = 'Target'

In [36]:
root = tree4.fit(data,features,target)

0.9402859586706311
1.5774062828523454
{64: 0.8925768472426705, 65: 0.9299678577609909, 70: 0.8949517866414867, 71: 0.9389462162661898, 72: 0.9389462162661898, 75: 0.9152077851647805, 80: 0.9397964894836077, 83: 0.8268850944895277}
Best Cut point: 0.8268850944895277 and 83
0.3712323266408755
{65: 0.8925768472426705, 70: 0.9253298887416583, 80: 0.7884504573082896, 85: 0.8921589282623617, 86: 0.8380423950607804, 90: 0.8609819860721621, 95: 0.8925768472426705}
Best Cut point: 0.7884504573082896 and 80
1.0
0.9852281360342515
[0.15642756 0.30547142 0.1518355  0.04884862]
0.8904916402194913
1.5766212201074912
{65: 0.847657692973375, 70: 0.8853673080251491, 86: 0.707943732143855, 90: 0.7672437270028533, 95: 0.847657692973375}
Best Cut point: 0.707943732143855 and 86
0.9612366047228759
0.9957274520849255
[0.13278826 0.18990944 0.11083369]
0.5435644431995964
1.561278124459133
0.954434002924965
[0.12759002 0.20871376]
0.9182958340544896
1.584962500721156
[0.57938016]
0.9709505944546686
1.52192809

In [37]:
tree4.viewTree(root)

Depth: 0 Attribute: Temperature
[1: (83, 0): Attribute: Humidity]  [2: (83, 1): Class label: No]  

Depth: 1 Attribute: Humidity
[1: (86, 0): Attribute: Wind]  [2: (86, 1): Attribute: Outlook]  

Depth: 2 Attribute: Wind
[1: Strong: Attribute: Outlook]  [2: Weak: Class label: Yes]  

Depth: 2 Attribute: Outlook
[1: Overcast: Class label: Yes]  [2: Rain: Attribute: Wind]  [3: Sunny: Class label: No]  

Depth: 3 Attribute: Outlook
[1: Overcast: Class label: Yes]  [2: Rain: Class label: No]  [3: Sunny: Class label: Yes]  

Depth: 3 Attribute: Wind
[1: Strong: Class label: No]  [2: Weak: Class label: Yes]  



In [38]:
pred_y = tree4.predict(root,data.iloc[:,:-1])

In [39]:
tree4.accuracy(data.iloc[:,-1].values,pred_y)

100.0

In [40]:
chess = pd.read_csv('krkopt.csv')

In [41]:
chess['1'] = chess['1'].astype(np.object)
chess['2'] = chess['2'].astype(np.object)
chess['3'] = chess['3'].astype(np.object)

In [42]:
chess['1'] = chess['1'].apply(lambda x:str(x))
chess['2'] = chess['2'].apply(lambda x:str(x))
chess['3'] = chess['3'].apply(lambda x:str(x))

In [43]:
train_c,test_c = train_test_split(chess,test_size=0.2,random_state=0)

In [45]:
tree_chess = C45()

In [46]:
features = train_c.columns[:-1].to_list()

In [47]:
root_chess = tree_chess.fit(train_c,features,'draw')

3.5073399760327666
1.7638082511471596
1.739186232712856
2.9995749076680553
2.9998805302152824
2.9520489142404718
2.996556083685621
[0.09467124 0.16767409 0.01685562 0.01529362 0.06346905 0.10472328]
3.136813374375866
1.957072995427066
2.9997117640891906
2.999017806537227
2.977792630030746
2.9899153525409146
[0.06031835 0.03822472 0.02979177 0.08634439 0.10103506]
3.6844532896660427
1.99354969719945
2.998778162774994
2.99375198150493
2.841606896782485
[0.33055642 0.12709888 0.16294414 0.21532797]
3.0501496143900084
2.992937636337307
2.9937405455921113
2.5836090721249514
[0.22057411 0.301097   0.13050514]
2.5658382453124315
2.765535941613833
2.581358330499424
[0.30849065 0.33413136]
1.5219280948873621
2.321928094887362
[0.65545875]
2.2516291673878226
2.584962500721156
[0.87104906]
1.5219280948873621
2.321928094887362
[0.65545875]
1.5219280948873621
2.321928094887362
[0.65545875]
1.3709505944546687
2.321928094887362
[0.59043628]
1.9219280948873623
2.321928094887362
[0.82772938]
1.70563431

In [48]:
preds = tree_chess.predict(root_chess,test_c.iloc[:,:-1])

In [49]:
tree_chess.accuracy(test_c.iloc[:,-1].values,preds)

59.04473355908038